In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
DATA_PATH = './data/'
LIGHTCURVES_PATH = DATA_PATH + 'lightcurves/'
FEATURES_PATH = DATA_PATH + 'features/'

In [3]:
import numpy as np
import pandas as pd
import measurements, extract
import matplotlib.pyplot as plt
import inputs2
from log_progress import log_progress

In [4]:
def unique_ids_list(df_lcs):
    return df_lcs.index.get_level_values('ID').unique().format()

def print_num_ids_shape(df_lcs):
    unique_ids = unique_ids_list(df_lcs)
    print('Num IDs: {}  Shape: {}'.format(len(unique_ids), df_lcs.shape))

#### Import

Import __supernova__ lightcurves

In [5]:
filename = 'sns_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
sns = pd.read_pickle(filepath)
print_num_ids_shape(sns)

Num IDs: 5000  Shape: (186510, 4)


Import __non-supernovas__ light curves

In [6]:
filename = 'not_sns_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
nsns = pd.read_pickle(filepath)
print_num_ids_shape(nsns)

Num IDs: 172189  Shape: (8580130, 4)


In [7]:
sns.head()


MJD BANDPASS      FLUX  FLUX_ERROR
ID   observation_id                                          
SN_0 0               53094.543        i   18.7260       2.064
     1               53094.586        r    7.3375       1.673
     2               53094.613        z    9.8390       9.255
     3               53109.492        i  125.7100       3.098
     4               53109.539        r  110.4600       2.639

#### Add class and set BANDPASS as index


__SN__

In [8]:
sns['class'] = 'sn'

__Non-SN__

In [9]:
nsns['class'] = 'non-sn'

In [10]:
sns = sns.set_index(['BANDPASS'], append=True)
nsns = nsns.set_index(['BANDPASS'], append=True)

In [11]:
sns.head()

MJD      FLUX  FLUX_ERROR class
ID   observation_id BANDPASS                                       
SN_0 0              i         53094.543   18.7260       2.064    sn
     1              r         53094.586    7.3375       1.673    sn
     2              z         53094.613    9.8390       9.255    sn
     3              i         53109.492  125.7100       3.098    sn
     4              r         53109.539  110.4600       2.639    sn

#### Filter

In [12]:
sns.head()

MJD      FLUX  FLUX_ERROR class
ID   observation_id BANDPASS                                       
SN_0 0              i         53094.543   18.7260       2.064    sn
     1              r         53094.586    7.3375       1.673    sn
     2              z         53094.613    9.8390       9.255    sn
     3              i         53109.492  125.7100       3.098    sn
     4              r         53109.539  110.4600       2.639    sn

# Make columns index again and then run the filter

In [19]:
def filter_light_curves(sns, min_obs):
#     df_count = df_lcs.groupby(['ID','BANDPASS'], as_index=True).count()
#     df_count['ObsCount'] = df_count['FLUX']
#     df_count = df_count[['ObsCount']]
# #     df_count.head()
#     df_lcs_with_counts = df_lcs.join(df_count, how='inner')
#     # Remove objects with less than min_obs
#     df_filtered = df_lcs_with_counts[df_lcs_with_counts.ObsCount >= min_obs]
    
    df_count = sns.groupby(['ID','BANDPASS'], as_index=True).count()
    df_count['ObsCount'] = df_count['FLUX']
    df_count = df_count[['ObsCount']]

    sns = sns.reset_index(level=['ID', 'BANDPASS'])
    df_count = df_count.reset_index(level=['ID', 'BANDPASS'])

    df_lcs_with_counts = sns.merge(df_count,on=['ID','BANDPASS'])

    df_lcs_with_counts.head()

    df_lcs_with_counts.index.name = 'observation_id'
    df_lcs_with_counts = df_lcs_with_counts.set_index(['ID'], append=True)
    df_lcs_with_counts = df_lcs_with_counts.set_index(['BANDPASS'], append=True)
    df_lcs_with_counts = df_lcs_with_counts.reorder_levels(['ID', 'observation_id','BANDPASS'])
#     # Remove ObsCount
    df_filtered = df_lcs_with_counts[df_lcs_with_counts.ObsCount >= min_obs]
    df_filtered = df_filtered.drop(['ObsCount'], axis=1)
# 

    return df_filtered

def merge_multi(self, df, on):
    return self.reset_index().join(df,on=on,how=inner).set_index(self.index.names)

def sample(df_lcs, num_samples):
    # Set random seed
    np.random.seed(42)
    # Sample non-transient subset of same size as transients
    IDs = np.random.choice(unique_ids_list(df_lcs), size=num_samples, replace=False)
#     print(IDs); return
    df_sampled = df_nont.loc[IDs]
    return df_sampled

Filter __supernova__ light curves

In [20]:
df_sn_5 = filter_light_curves(sns, 5)
print_num_ids_shape(df_sn_5)

del sns

Num IDs: 5000  Shape: (179730, 4)


In [23]:
df_sn_5.head()

MJD     FLUX  FLUX_ERROR class
ID   observation_id BANDPASS                                      
SN_0 0              i         53094.543   18.726       2.064    sn
     1              i         53109.492  125.710       3.098    sn
     2              i         53112.527  139.860       4.138    sn
     3              i         53116.500  151.710       3.352    sn
     4              i         53117.441  156.310       4.463    sn

Filter __non-supernova__ lightcurves

In [24]:
df_nsns_5 = filter_light_curves(nsns, 5)
print_num_ids_shape(df_nsns_5)

del nsns

NameError: name 'nsns' is not defined

#### Oversample - we dont need this - set copies to 0 

In [25]:
def oversample(df_lcs, copies=0):
    df_oversample = df_lcs.copy()
    df_oversample['copy_num'] = 0
    for i in range(1, copies+1):
        df_temp = df_lcs.copy()
        df_temp['copy_num'] = i
        df_temp['Mag'] = np.random.normal(df_lcs.Mag, df_lcs.Magerr)
        df_oversample = df_oversample.append(df_temp)
        
    df_oversample = df_oversample.set_index(['copy_num'], append=True)
    return df_oversample

Oversample __supernovas__ light curves

In [26]:
df_sns_0_os = oversample(df_sn_5, 0)
print_num_ids_shape(df_sns_0_os)

del df_sn_5

Num IDs: 5000  Shape: (179730, 4)


Oversample __non-supernovas__ light curves

In [27]:
df_nsns_0_os = oversample(df_nsns_5, 0)
print_num_ids_shape(df_nsns_0_os)

del df_nsns_5

Num IDs: 161855  Shape: (7806031, 4)


Add bandpass as index

#### Feature Extraction

In [16]:
def extract_features(df_lcs):
    # Extract num_copy list
    num_copy_list = df_lcs.index.get_level_values('copy_num').unique()    
    num_copies = len(num_copy_list)
    
    # Extract IDs list
    unique_ids_list = df_lcs.index.get_level_values('ID').unique()
    num_ids = len(unique_ids_list)

    # Create empty feature dict
    feats_dict = extract.feature_dict(30)
    feats_dict['ObsCount'] = []
    feats_dict['Class'] = []
    # Add 'ID' and 'copy_num' index lists
    index_id_list = []
    index_copy_num_list = []
    
    num_objects = num_ids*num_copies
    for num_copy in num_copy_list:
        for i, obj_id in enumerate(unique_ids_list):
            # Print status
            current_object_i = (num_copy+1)*i
            if(current_object_i%int(num_objects/10) == 0):
                print(current_object_i, '/', num_objects)
            # Get current object light curve
            df_object = df_lcs.loc[obj_id,:,num_copy]
            # Get features
            obj_feats = extract.features(df_object, feats_dict)
            # Append features
            for k,v in obj_feats.items():
                feats_dict[k].append(obj_feats[k])
            # Append Indexes
            index_id_list.append(obj_id)
            index_copy_num_list.append(num_copy)
            # Append class and obs_count
            assert(len(df_object['class'].unique()) == 1)
            assert(len(df_object['ObsCount'].unique()) == 1)
            assert(df_object['ObsCount'].unique()[0] == df_object.shape[0])
            feats_dict['Class'].append(df_object['class'].unique()[0])
            feats_dict['ObsCount'].append(df_object.shape[0])
            
            
    # Create feature dataframe
    df_feats = pd.DataFrame(feats_dict).set_index([index_id_list,index_copy_num_list])
    df_feats.index.names = ['ID', 'copy_num']
    return df_feats

def save_features(df_feats, obj_type):
    outdir = FEATURES_PATH
    filename_raw = '{}.pickle'
    filename = filename_raw.format(obj_type)
    assert(df_feats.shape[1]==32) # 30 + ['num_obs'+'class']
    df_feats.to_pickle(outdir + filename)

#### Generate Features

In [17]:
def generate_features(df_all, sn, min_obs):
    obj_type = 'SN' if sn else 'NSN'
    
    # Generate features based on light curves
    df_feats = extract_features(df_all)
    
    # Save features
    save_features(df_feats, obj_type)
    
    # Log Finished
    print('Finished task type={} obs={}'.format(obj_type, min_obs) )
    
    return df_feats

Generate features __transient__ light curves

In [18]:
df_tra_feats = generate_features(df_tra_5_os, transient=True, min_obs=5)
df_nont_feats = generate_features(df_nont_5_os, transient=False, min_obs=5)

0 / 46959


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks

0 / 46959


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks

0 / 46959
4695 / 46959


...

Finished task type=T obs=5
0 / 15193
1519 / 15193
3038 / 15193
4557 / 15193
6076 / 15193
7595 / 15193


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)


9114 / 15193
10633 / 15193
12152 / 15193
13671 / 15193


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)


15190 / 15193
Finished task type=NT obs=5


In [38]:
df_sns_0_os.head()

MJD      FLUX  FLUX_ERROR class  \
ID   observation_id copy_num BANDPASS                                          
SN_0 0              0        i         53094.543   18.7260       2.064    sn   
     1              0        r         53094.586    7.3375       1.673    sn   
     2              0        z         53094.613    9.8390       9.255    sn   
     3              0        i         53109.492  125.7100       3.098    sn   
     4              0        r         53109.539  110.4600       2.639    sn   

                                       ObsCount  
ID   observation_id copy_num BANDPASS            
SN_0 0              0        i               60  
     1              0        r               60  
     2              0        z               60  
     3              0        i               60  
     4              0        r               60

In [61]:
num_copy_list = df_sns_0_os.index.get_level_values('copy_num').unique()    
num_copies = len(num_copy_list)

In [62]:
# Extract IDs list
unique_ids_list = df_sns_0_os.index.get_level_values('ID').unique()
num_ids = len(unique_ids_list)

In [63]:
# Extract bandpass list
unique_bandpass_list = df_sns_0_os.index.get_level_values('BANDPASS').unique()
num_bandpass = len(unique_bandpass_list)

In [64]:
# Create empty feature dict
feats_dict = extract.feature_dict(30)
feats_dict['ObsCount'] = []
feats_dict['Class'] = []

In [65]:
# Add 'ID' and 'copy_num' index lists
index_id_list = []
index_copy_num_list = []
index_bandpass_list =  []

In [69]:
num_objects = num_ids*num_copies*num_bandpass
for num_copy in num_copy_list:
    for i, obj_id in enumerate(unique_ids_list):
        for j, bandpass_id in enumerate(unique_bandpass_list):

            # Print status
            current_object_i = (num_copy+1)*i + j
            if(current_object_i%int(num_objects/10) == 0):
                print(current_object_i, '/', num_objects)

            # Get current object light curve
            df_object = df_sns_0_os.loc[obj_id,:,num_copy,bandpass_id]

            # Get features
            obj_feats = extract.features(df_object, feats_dict)

            # Append features
            for k,v in obj_feats.items():
                feats_dict[k].append(obj_feats[k])

            # Append Indexes
            index_id_list.append(obj_id)
            index_copy_num_list.append(num_copy)
            index_bandpass_list.append(bandpass_id)

            # Append class and obs_count
            assert(len(df_object['class'].unique()) == 1)
            assert(len(df_object['ObsCount'].unique()) == 1)
#             assert(len(df_object['BANDPASS'].unique()) == 1)
#             assert(df_object['ObsCount'].unique()[0] == df_object.shape[0])
            feats_dict['Class'].append(df_object['class'].unique()[0])
            feats_dict['ObsCount'].append(df_object.shape[0])


# Create feature dataframe
df_feats = pd.DataFrame(feats_dict).set_index([index_id_list,index_copy_num_list,index_bandpass_list])
df_feats.index.names = ['ID', 'copy_num','BANDPASS']
# return df_feats

0 / 20000


/home/mauro/Documents/U/monitoriaInv/monInv/measurements.py:218: RankWarning: Polyfit may be poorly conditioned
  p2 = np.polyfit(x, y, 2)
/home/mauro/Documents/U/monitoriaInv/monInv/measurements.py:219: RankWarning: Polyfit may be poorly conditioned
  p3 = np.polyfit(x, y, 3)
/home/mauro/Documents/U/monitoriaInv/monInv/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)


ZeroDivisionError: float division by zero

In [46]:
df_sns_0_os.head()

MJD      FLUX  FLUX_ERROR class  \
ID   observation_id copy_num BANDPASS                                          
SN_0 0              0        i         53094.543   18.7260       2.064    sn   
     1              0        r         53094.586    7.3375       1.673    sn   
     2              0        z         53094.613    9.8390       9.255    sn   
     3              0        i         53109.492  125.7100       3.098    sn   
     4              0        r         53109.539  110.4600       2.639    sn   

                                       ObsCount  
ID   observation_id copy_num BANDPASS            
SN_0 0              0        i               60  
     1              0        r               60  
     2              0        z               60  
     3              0        i               60  
     4              0        r               60

In [71]:
float(df_sns_0_os.FLUX.shape[0])

186510.0

In [73]:
temp = df_sns_0_os.loc["SN_0",:,0,'i']

In [75]:
float(temp.FLUX.shape[0])

21.0